###Silver Layer

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.Silver
""")

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.Silver.ecommerce_data
""")

In [0]:
import pyspark.sql.functions as F

In [0]:
# SILVER: Cleaned data
bronze = spark.table("workspace.bronze.ecommerce_data")
display(bronze)

In [0]:
silver = bronze.filter(F.col("price") > 0) \
    .filter(F.col("price") < 10000) \
    .dropDuplicates(["user_session", "event_time"]) \
    .withColumn("event_date", F.to_date("event_time")) \
    .withColumn("price_tier",
        F.when(F.col("price") < 10, "budget")
         .when(F.col("price") < 50, "mid")
         .otherwise("premium"))
silver.write.format("delta").mode("overwrite").saveAsTable("workspace.Silver.ecommerce_data")    
#silver.write.format("delta").mode("overwrite").save("/Volumes/workspace/silver/ecommerce_data")

In [0]:
silver.display()

In [0]:
silver.count()